# LLM-Powered AI Agents for Building and Validating Computer Vision Pipelines

## Install required libraries

In [1]:
!pip install -q -U pynvml bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

## Loading the required libraries

In [2]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig,
    set_seed
)
from tqdm import tqdm
from trl import SFTTrainer
from huggingface_hub import interpreter_login
from pynvml import *
from functools import partial
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
import time
import pandas as pd
import numpy as np
import transformers
import evaluate

In [3]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [4]:
# hugging face login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


## GPU Utilization

In [5]:
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")

## Loading dataset

In [6]:
df=pd.read_csv("./cva_gstreamer_pipeline_dataset_V1.csv")

In [7]:
dataset = Dataset.from_pandas(df)

In [8]:
dataset[0]

{'prompt': '{"source_type": "rtsp", "source_location": "rtsp://192.168.1.124:8069/preview", "video_scale": {"width": 1597, "height": 518}, "caps_filter": "GBR", "detect": {"model": "/usr/share/models/onnx/detection/1.0/FP32/model_8007.xml", "device": "HDDL"}, "track": {"tracking_type": 0}, "classify": {"model": "/opt/models/intel/segmentation/2.0/FP32/model_3695.xml", "device": "GPU", "inference_region": "full-frame", "model_proc": "/home/user/.local/intel/model_proc/detection/model_3956.json"}, "inference": [{"model": "/home/user/.local/models/intel/classification/5.0/FP32/model_5775.xml", "device": "CPU", "inference_region": "roi-list"}, {"model": "/usr/share/models/onnx/recognition/2.1/FP16/model_7317.xml", "device": "HDDL", "inference_region": "roi-list"}, {"model": "/opt/models/intel/detection/3.0/FP16/model_9971.xml", "device": "CPU", "inference_region": "roi-list"}, {"model": "/usr/share/models/intel/segmentation/5.0/INT8/model_6368.xml", "device": "CPU", "inference_region": "ro

## Test Train Validation Split

In [9]:
train_test = dataset.train_test_split(test_size=0.2, seed=42)

In [10]:
test_validation = train_test['test'].train_test_split(test_size=0.5, seed=42)

In [11]:
dataset = {
    "train": train_test['train'],
    "test": test_validation['test'],
    "validation": test_validation['train']
}

## Create Bitsandbytes configuration

In [12]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

device_map = {"": 0}

## Loading the Pre-Trained model

In [13]:
model_name='microsoft/phi-2'
base_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Tokenization

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [15]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [16]:
def generate_text_from_model(model,prompt):
    toks = eval_tokenizer(prompt, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=1000, do_sample=True,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

## Test the Model with Zero Shot Inferencing

In [17]:
def get_formatted_prompt(prompt):
    return f"Task: Convert the JSON configuration below into a valid gst-launch-1.0 pipeline command. Ensure all elements and properties are properly formatted.\nInput JSON:\n{prompt}\nOutput:\n"

In [18]:
%%time
seed = 42
set_seed(seed)

index = 10

sample_prompt = dataset['test'][index]['prompt']
sample_pipeline = dataset['test'][index]['pipeline']

formatted_prompt = get_formatted_prompt(sample_prompt)
res = generate_text_from_model(base_model,formatted_prompt)
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'GROUND TRUTH:\n{sample_pipeline}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Task: Convert the JSON configuration below into a valid gst-launch-1.0 pipeline command. Ensure all elements and properties are properly formatted.
Input JSON:
{"source_type": "file", "source_location": "/opt/warehouse_8/aisle/sensor_3.mp4", "video_crop": {"top": 496, "left": 71, "right": 67, "bottom": 129}, "caps_filter": "NV12_16L32S", "frame_rate": "50/2", "detect": {"model": "/usr/local/models/onnx/detection/3.0/INT8/model_9162.xml", "device": "HDDL", "threshold": 0.76}, "track": {"tracking_type": "zero-term-imageless"}, "emit_signals": false, "sync": false, "drop": true}
Output:

---------------------------------------------------------------------------------------------------
GROUND TRUTH:
gst-launch-1.0 filesrc location=/opt/warehouse_8/aisle/sensor_3.mp4 ! decodebin ! videoconvert ! videorate ! video/x-raw,framerate=50/2 ! videoconvert ! capsfilter caps=video/x-raw

## Pre-processing dataset

In [19]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "This tool generates GStreamer pipeline commands from JSON configurations. Follow the input format to create valid gst-launch-1.0 commands."
    INSTRUCTION_KEY = "### Instruct: Convert the JSON configuration below into a valid gst-launch-1.0 pipeline command. Ensure all elements and properties are properly formatted."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['prompt']}" if sample["prompt"] else None
    response = f"{RESPONSE_KEY}\n{sample['pipeline']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    sample["text"] = "\n\n".join(parts)

    return sample

In [20]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

In [21]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats) #, batched=True)
    
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    
    dataset = dataset.map(
        _preprocessing_function,
        batched=True
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [22]:
print_gpu_utilization()

GPU memory occupied: 2631 MB.


In [23]:
## Pre-process dataset
max_length = get_max_length(base_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Filter:   0%|          | 0/800 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [24]:
print(f"Shapes of the datasets:")
print(f"Training: {train_dataset.shape}")
print(f"Validation: {eval_dataset.shape}")
print(train_dataset)

Shapes of the datasets:
Training: (800, 5)
Validation: (100, 5)
Dataset({
    features: ['prompt', 'pipeline', 'text', 'input_ids', 'attention_mask'],
    num_rows: 800
})


## Setup PEFT for Fine-Tuning

In [25]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(base_model))

trainable model parameters: 262364160
all model parameters: 1521392640
percentage of trainable model parameters: 17.24%


In [26]:
print(base_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_la

In [27]:
config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
base_model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
base_model = prepare_model_for_kbit_training(base_model)

peft_model = get_peft_model(base_model, config)

In [28]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [29]:
print(peft_model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleD

## Train PEFT Adapter

In [30]:
output_dir = './peft_GST_pipeline_training_QLora/final-checkpoint'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=100,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=5,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=5,
    eval_strategy="steps",
    eval_steps=5,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [31]:
peft_training_args.device

device(type='cuda', index=0)

In [32]:
peft_trainer.train()

Step,Training Loss,Validation Loss
5,1.296200,1.575799
10,1.235000,1.291856
15,1.011300,0.973989
20,0.834600,0.702366
25,0.707000,0.511820
30,0.547900,0.407002
35,0.440500,0.355748
40,0.413100,0.324753
45,0.354200,0.308422
50,0.321100,0.300194


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=100, training_loss=0.49751404762268064, metrics={'train_runtime': 1302.6041, 'train_samples_per_second': 0.307, 'train_steps_per_second': 0.077, 'total_flos': 3775436910213120.0, 'train_loss': 0.49751404762268064, 'epoch': 0.5})

In [33]:
print_gpu_utilization()

GPU memory occupied: 11077 MB.


In [49]:
# Free memory for merging weights
del base_model
del peft_trainer
torch.cuda.empty_cache()

NameError: name 'peft_trainer' is not defined

In [50]:
print_gpu_utilization()

GPU memory occupied: 5469 MB.


##  Evaluate the Model Qualitatively (Human Evaluation)

In [51]:
base_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [57]:
ft_model = PeftModel.from_pretrained(base_model, "peft_GST_pipeline_training_QLora/final-checkpoint/checkpoint-100",torch_dtype=torch.float16,is_trainable=False)

In [58]:
%%time
seed = 42
set_seed(seed)

index = 10

sample_prompt = dataset['test'][index]['prompt']
sample_pipeline = dataset['test'][index]['pipeline']

formatted_prompt = get_formatted_prompt(sample_prompt)

peft_model_res = generate_text_from_model(ft_model,formatted_prompt)
peft_model_output = peft_model_res[0].split('Output:\n')[1]
prefix, success, result = peft_model_output.partition('#End')

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}\n')
print(dash_line)
print(f'GROUND TRUTH:\n{sample_pipeline}\n')
print(dash_line)
print(f'PEFT MODEL:\n{prefix}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Task: Convert the JSON configuration below into a valid gst-launch-1.0 pipeline command. Ensure all elements and properties are properly formatted.
Input JSON:
{"source_type": "file", "source_location": "/opt/warehouse_8/aisle/sensor_3.mp4", "video_crop": {"top": 496, "left": 71, "right": 67, "bottom": 129}, "caps_filter": "NV12_16L32S", "frame_rate": "50/2", "detect": {"model": "/usr/local/models/onnx/detection/3.0/INT8/model_9162.xml", "device": "HDDL", "threshold": 0.76}, "track": {"tracking_type": "zero-term-imageless"}, "emit_signals": false, "sync": false, "drop": true}
Output:


---------------------------------------------------------------------------------------------------
GROUND TRUTH:
gst-launch-1.0 filesrc location=/opt/warehouse_8/aisle/sensor_3.mp4 ! decodebin ! videoconvert ! videorate ! video/x-raw,framerate=50/2 ! videoconvert ! capsfilter caps=video/x-ra

## Evaluate the Model Quantitatively (with ROUGE Metric)

In [66]:
base_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [72]:
prompts = dataset['test'][0:10]['prompt']
target_pipelines = dataset['test'][0:10]['pipeline']

base_model_pipelines = []
peft_model_pipelines = []

for idx, prompt in enumerate(prompts):
    target_text_output = target_pipelines[idx]
    formatted_prompt = get_formatted_prompt(prompt)
    
    base_model_res = generate_text_from_model(base_model,formatted_prompt)
    base_model_text_output = base_model_res[0].split('Output:\n')[1]
    
    peft_model_res = generate_text_from_model(ft_model,formatted_prompt)
    peft_model_output = peft_model_res[0].split('Output:\n')[1]

    peft_model_text_output, success, result = peft_model_output.partition('#End')
    

    base_model_pipelines.append(base_model_text_output)
    peft_model_pipelines.append(peft_model_text_output)

zipped_pipelines = list(zip(target_pipelines, base_model_pipelines, peft_model_pipelines))
 
df = pd.DataFrame(zipped_pipelines, columns = ['target_pipelines', 'base_model_pipelines', 'peft_model_pipelines'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [73]:
df

,target_pipelines,base_model_pipelines,peft_model_pipelines
0,gst-launch-1.0 rtspsrc location=rtsp://192.168...,gst-launch-1.0 -s source_type=rtsp -s source_l...,gst-launch-1.0 rtspsrc location=rtsp://192.168...
1,gst-launch-1.0 filesrc location=/opt/retail/sh...,gst-launch-1.0 -source_type file -source_locat...,gst-launch-1.0 filesrc location=/opt/retail/sh...
2,gst-launch-1.0 rtspsrc location=rtsp://192.168...,gst-launch-1.0 -s source_type=rtsp -s source_l...,gst-launch-1.0 rtspsrc location=rtsp://192.168...
3,gst-launch-1.0 filesrc location=/opt/mall_6/ai...,gst-launch-1.0 -source_type file -source_locat...,gst-launch-1.0 filesrc location=/opt/mall_6/ai...
4,gst-launch-1.0 filesrc location=/opt/data/shop...,gst-launch-1.0 -s source_type=file -s source_l...,gst-launch-1.0 filesrc location=/opt/data/shop...
5,gst-launch-1.0 filesrc location=/opt/data/shop...,gst-launch-1.0 -s source_type=file -s source_l...,gst-launch-1.0 filesrc location=/opt/data/shop...
6,gst-launch-1.0 filesrc location=/opt/shop_8/ai...,gst-launch-1.0 -s source_type=file -s source_l...,gst-launch-1.0 filesrc location=/opt/shop_8/ai...
7,gst-launch-1.0 rtspsrc location=rtsp://192.168...,gst-launch-1.0 -s source_type=rtsp -s source_l...,gst-launch-1.0 rtspsrc location=rtsp://192.168...
8,gst-launch-1.0 filesrc location=/opt/data/reta...,gst-launch-1.0 -s source_type=file -s source_l...,gst-launch-1.0 filesrc location=/opt/data/reta...
9,gst-launch-1.0 filesrc location=/opt/data/reta...,gst-launch-1.0 -s source_type=file -s source_l...,gst-launch-1.0 filesrc location=/opt/data/reta...


In [74]:
rouge = evaluate.load('rouge')

base_model_results = rouge.compute(
    predictions=base_model_pipelines,
    references=target_pipelines[0:len(base_model_pipelines)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_pipelines,
    references=target_pipelines[0:len(peft_model_pipelines)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(base_model_results)
print('PEFT MODEL:')
print(peft_model_results)


ORIGINAL MODEL:
{'rouge1': np.float64(0.738271826036321), 'rouge2': np.float64(0.6168275504132683), 'rougeL': np.float64(0.6990100149374439), 'rougeLsum': np.float64(0.6964834392441597)}
PEFT MODEL:
{'rouge1': np.float64(0.9778220179858674), 'rouge2': np.float64(0.9685361200048618), 'rougeL': np.float64(0.9727569471257282), 'rougeLsum': np.float64(0.9734316296669598)}


In [75]:
print("Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(base_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over ORIGINAL MODEL
rouge1: 23.96%
rouge2: 35.17%
rougeL: 27.37%
rougeLsum: 27.69%


In [76]:
df.to_csv("peft_results.csv", index=False, header=True)